In [ ]:
!pip install transformers[torch]
!pip install sentencepiece
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00


In [ ]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration
import accelerate
import bitsandbytes as bnb

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto", load_in_8bit=True)



tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00005.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/6.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import numpy as np
import torch

player_stats = pd.read_csv('players_stats.csv')
vorp = pd.read_csv('vorp_binary.csv')

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(player_stats, vorp, test_size=0.25, random_state=42)

train_data.index = range(len(train_data))
train_labels.index = range(len(train_labels))
test_data.index = range(len(test_data))
test_labels.index = range(len(test_labels))


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(player_stats, vorp, test_size=0.25, random_state=42)

train_data.index = range(len(train_data))
train_labels.index = range(len(train_labels))
test_data.index = range(len(test_data))
test_labels.index = range(len(test_labels))


training_inputs = []
for index, row in train_data.iterrows():
  # Will include more in future
  train_input = f"{row['Player']} has Points: {row['Points']}, Rebounds: {row['Rebounds']}, Assists: {row['Assists']}, Steals: {row['Steals']}, Blocks: {row['Blocks']}"
  training_inputs.append(train_input)

training_labels = []
for _,row in train_labels.iterrows():
  # Will include more in future
  train_label = f"{row['vorp']}"
  training_labels.append(int(train_label))


# Initialize empty lists for test inputs and labels
testing_inputs = []
testing_labels = []

# Loop through test_data for inputs
for index, row in test_data.iterrows():
    # Will include more in the future
    test_input = f"{row['Player']} has Points: {row['Points']}, Rebounds: {row['Rebounds']}, Assists: {row['Assists']}, Steals: {row['Steals']}, Blocks: {row['Blocks']}"
    testing_inputs.append(test_input)

# Loop through test_labels for labels
for _, row in test_labels.iterrows():
    # Will include more in the future
    test_label = f"{row['vorp']}"
    testing_labels.append(int(test_label))

In [ ]:



base_prompt = """
Is VORP Positive or Negative? Here are three examples:
Zaid Abdul-Aziz has Points: 9.0, Rebounds: 8.0, Assists: 1.2, Steals: 0.6, Blocks: 1.0. VORP is Positive.
Alex Acker has Points: 2.7, Rebounds: 1.0, Assists: 0.5, Steals: 0.2, Blocks: 0.1. VORP is Negative.
Billy Paultz has Points: 21.7, Rebounds: 8.0, Assists: 2.0, Steals: 0.5, Blocks: 1.5. VORP is Positive.
"""


prompt_inputs = [base_prompt + i + "." for i in testing_inputs]

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


count = 0
total = 0
batch_size = 64


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.generate(**t, max_new_tokens = 10).detach().cpu()
    decoded_texts = tokenizer.batch_decode(pred, skip_special_tokens=True)



    for k in range(len(decoded_texts)):


      if "positive" in decoded_texts[k].lower() or "pos" in decoded_texts[k].lower():
        predictions.append(1)
      elif "negative" in decoded_texts[k].lower() or "neg" in decoded_texts[k].lower():
        predictions.append(0)

      else:
        print("ERROR")



print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

0.8204134366925064
0.818639798488665
0.8290816326530612
0.8238276299112801


In [ ]:

base_prompt = """
Is VORP Positive or Negative? Here are three examples:
Zaid Abdul-Aziz has Points: 9.0, Rebounds: 8.0, Assists: 1.2, Steals: 0.6, Blocks: 1.0. VORP (value over replacement) is Positive because Points, Rebounds, and Blocks are above the mean - mean Points: 7.6, mean Rebounds: 3.4, mean Assists: 1.7, mean Steals: 0.6, mean blocks: 0.4.
Alex Acker has Points: 2.7, Rebounds: 1.0, Assists: 0.5, Steals: 0.2, Blocks: 0.1. VORP (value over replacement) is Negative because Points, Rebounds, Assists, Steals and Blocks are below the mean - mean Points: 7.6, mean Rebounds: 3.4, mean Assists: 1.7, mean Steals: 0.6, mean blocks: 0.4.
Billy Paultz has Points: 21.7, Rebounds: 8.0, Assists: 2.0, Steals: 0.5, Blocks: 1.5. VORP (value over replacement) is Positive because Points, Rebounds, Assists, and Blocks are below the mean - mean Points: 7.6, mean Rebounds: 3.4, mean Assists: 1.7, mean Steals: 0.6, mean blocks: 0.4.
"""

prompt_inputs = [base_prompt + i + "." for i in testing_inputs]

In [ ]:
count = 0
total = 0
batch_size = 64


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.generate(**t, max_new_tokens = 10).detach().cpu()
    decoded_texts = tokenizer.batch_decode(pred, skip_special_tokens=True)




    for k in range(len(decoded_texts)):


      if "positive" in decoded_texts[k].lower() or "pos" in decoded_texts[k].lower():
        predictions.append(1)
      elif "negative" in decoded_texts[k].lower() or "neg" in decoded_texts[k].lower():
        predictions.append(0)

      else:
        print("ERROR")



print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

0.8229974160206718
0.836272040302267
0.8217821782178217
0.8289637952559301


In [ ]:
count = 0
total = 0
batch_size = 16


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred1 = model.generate(**t, max_new_tokens = 10).detach().cpu()
    pred2 = model.generate(**t, max_new_tokens = 10, do_sample= True, top_k = 3).detach().cpu()
    pred3 = model.generate(**t, max_new_tokens = 10, do_sample= True, top_k = 3).detach().cpu()




    decoded_texts1 = tokenizer.batch_decode(pred1, skip_special_tokens=True)
    decoded_texts2 = tokenizer.batch_decode(pred2, skip_special_tokens=True)
    decoded_texts3 = tokenizer.batch_decode(pred3, skip_special_tokens=True)




    for k in range(len(decoded_texts1)):
      pos = 0


      if "positive" in decoded_texts1[k].lower() or "pos" in decoded_texts1[k].lower():
        pos += 1

      if "positive" in decoded_texts2[k].lower() or "pos" in decoded_texts2[k].lower():
        pos += 1

      if "positive" in decoded_texts3[k].lower() or "pos" in decoded_texts3[k].lower():
        pos += 1

      if pos > 1:
        predictions.append(1)
      else:
        predictions.append(0)





print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

0.7338501291989664
0.7556675062972292
0.7334963325183375
0.7444168734491315


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))

print(f1_score(y_true=testing_labels, y_pred=predictions))

0.7338501291989664
0.7556675062972292
0.7334963325183375
0.7444168734491315


In [ ]:
hof

,Player,Points,Rebounds,Assists,Steals,Blocks,hof
0,Alaa Abdelnaby,5.7,3.3,0.3,0.3,0.3,0.0
1,Zaid Abdul-Aziz,9.0,8.0,1.2,0.6,1.0,0.0
2,Kareem Abdul-Jabbar,24.6,11.2,3.6,0.9,2.6,1.0
3,Mahmoud Abdul-Rauf,14.6,1.9,3.5,0.8,0.1,0.0
4,Tariq Abdul-Wahad,7.8,3.3,1.1,0.8,0.4,0.0
...,...,...,...,...,...,...,...
3088,Stephen Zimmerman,1.2,1.8,0.2,0.1,0.3,0.0
3089,Paul Zipser,4.7,2.6,0.8,0.4,0.3,0.0
3090,Ante Žižić,6.0,3.9,0.6,0.2,0.4,0.0
3091,Jim Zoet,0.3,1.1,0.1,0.1,0.4,0.0


In [ ]:
hof = pd.read_csv("hof.csv")


In [ ]:
hof_labels = [int(i) for i in hof["hof"]]



In [ ]:
from sklearn.model_selection import train_test_split

hof = pd.read_csv("hof.csv")
hof_labels = [int(i) for i in hof["hof"]]

train_data, test_data, training_labels, testing_labels = train_test_split(player_stats, hof_labels, test_size=0.25, random_state=42)

train_data.index = range(len(train_data))
#train_labels.index = range(len(train_labels))
test_data.index = range(len(test_data))
#test_labels.index = range(len(test_labels))


training_inputs = []
for index, row in train_data.iterrows():
  # Will include more in future
  train_input = f"{row['Player']} has Points: {row['Points']}, Rebounds: {row['Rebounds']}, Assists: {row['Assists']}, Steals: {row['Steals']}, Blocks: {row['Blocks']}"
  training_inputs.append(train_input)


testing_inputs = []
# Loop through test_data for inputs
for index, row in test_data.iterrows():
    # Will include more in the future
    test_input = f"{row['Player']} has Points: {row['Points']}, Rebounds: {row['Rebounds']}, Assists: {row['Assists']}, Steals: {row['Steals']}, Blocks: {row['Blocks']}"
    testing_inputs.append(test_input)


In [ ]:
hof[hof["hof"] == 1]

,Player,Points,Rebounds,Assists,Steals,Blocks,hof
2,Kareem Abdul-Jabbar,24.6,11.2,3.6,0.9,2.6,1.0
52,Ray Allen,18.9,4.1,3.4,1.1,0.2,1.0
97,Tiny Archibald,18.8,2.3,7.4,1.1,0.1,1.0
155,Charles Barkley,22.1,11.7,3.9,1.5,0.8,1.0
169,Rick Barry,24.8,6.7,4.9,2.0,0.5,1.0
...,...,...,...,...,...,...,...
2912,Jo Jo White,17.2,4.0,4.9,1.3,0.2,1.0
2935,Lenny Wilkens,16.5,4.7,6.7,1.3,0.2,1.0
2937,Jamaal Wilkes,17.7,6.2,2.5,1.3,0.3,1.0
2940,Dominique Wilkins,24.8,6.7,2.5,1.3,0.6,1.0


In [ ]:
base_prompt = """
Is this player in the hall of fame? Here are three examples:
Tiny Archibald has Points: 18.8, Rebounds: 2.3, Assists: 7.4, Steals: 1.1, Blocks: 0.1. Yes he is in the hall of fame.
Alex Acker has Points: 2.7, Rebounds: 1.0, Assists: 0.5, Steals: 0.2, Blocks: 0.1. No he is not in the hall of fame.
Charles Barkley has Points: 22.1, Rebounds: 11.7, Assists: 3.9, Steals: 1.5, Blocks: 0.8. Yes he is in the hall of fame.
Rick Barry has Points: 24.8, Rebounds: 6.7, Assists: 4.9, Steals: 2.0, Blocks: 0.5. Yes he is in the hall of fame.
"""


prompt_inputs = [base_prompt + i + "." for i in testing_inputs]

In [ ]:
count = 0
total = 0
batch_size = 64


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.generate(**t, max_new_tokens = 10).detach().cpu()
    decoded_texts = tokenizer.batch_decode(pred, skip_special_tokens=True)



    for k in range(len(decoded_texts)):



      if "yes" in decoded_texts[k].lower():
        predictions.append(1)
      elif "no" in decoded_texts[k].lower():
        predictions.append(0)

      else:
        print("ERROR")



print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

0.9664082687338501
0.07142857142857142
1.0
0.13333333333333333


In [ ]:
base_prompt = """
Is this player in the hall of fame? Here are three examples:
Tiny Archibald has Points: 18.8, Rebounds: 2.3, Assists: 7.4, Steals: 1.1, Blocks: 0.1. Yes he is in the hall of fame because Points, Assists, and Steals are far above the mean - mean Points: 7.6, mean Rebounds: 3.4, mean Assists: 1.7, mean Steals: 0.6, mean blocks: 0.4.
Alex Acker has Points: 2.7, Rebounds: 1.0, Assists: 0.5, Steals: 0.2, Blocks: 0.1. No he is not in the hall of fame because Points, Rebounds, Assists, Steals and Blocks are far below the mean - mean Points: 7.6, mean Rebounds: 3.4, mean Assists: 1.7, mean Steals: 0.6, mean blocks: 0.4.
Charles Barkley has Points: 22.1, Rebounds: 11.7, Assists: 3.9, Steals: 1.5, Blocks: 0.8. Yes he is in the hall of fame because Points, Rebounds, Assists, Steals and Blocks are far above the mean - mean Points: 7.6, mean Rebounds: 3.4, mean Assists: 1.7, mean Steals: 0.6, mean blocks: 0.4.
"""


prompt_inputs = [base_prompt + i + "." for i in testing_inputs]

In [ ]:
count = 0
total = 0
batch_size = 64


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.generate(**t, max_new_tokens = 10).detach().cpu()
    decoded_texts = tokenizer.batch_decode(pred, skip_special_tokens=True)



    for k in range(len(decoded_texts)):



      if "yes" in decoded_texts[k].lower():
        predictions.append(1)
      elif "no" in decoded_texts[k].lower():
        predictions.append(0)

      else:
        print("ERROR")



print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

0.9664082687338501
0.07142857142857142
1.0
0.13333333333333333


In [ ]:
count = 0
total = 0
batch_size = 16


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred1 = model.generate(**t, max_new_tokens = 10, do_sample= True, top_k = 3).detach().cpu()
    pred2 = model.generate(**t, max_new_tokens = 10, do_sample= True, top_k = 3).detach().cpu()
    pred3 = model.generate(**t, max_new_tokens = 10, do_sample= True, top_k = 3).detach().cpu()




    decoded_texts1 = tokenizer.batch_decode(pred1, skip_special_tokens=True)
    decoded_texts2 = tokenizer.batch_decode(pred2, skip_special_tokens=True)
    decoded_texts3 = tokenizer.batch_decode(pred3, skip_special_tokens=True)




    for k in range(len(decoded_texts1)):
      pos = 0


      if "yes" in decoded_texts1[k].lower():
        pos += 1

      if "yes" in decoded_texts2[k].lower():
        pos += 1

      if "yes" in decoded_texts3[k].lower():
        pos += 1

      if pos > 1:
        predictions.append(1)
      else:
        predictions.append(0)





print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

NameError: ignored

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

0.9599483204134367
0.10714285714285714
0.3333333333333333
0.16216216216216214


     Year                 Team   Age   MOV   SOS   SRS   ORtg   DRtg  NRtg  \
0  1990.0       Boston Celtics  30.2  3.99 -0.76  3.23  112.0  107.9   4.1   
1  1990.0        Chicago Bulls  26.1  3.26 -0.51  2.74  112.3  109.0   3.3   
2  1990.0  Cleveland Cavaliers  26.1 -0.30 -0.31 -0.62  106.9  107.2  -0.3   
3  1990.0     Dallas Mavericks  29.2  0.07  0.35  0.42  107.2  107.2   0.0   
4  1990.0       Denver Nuggets  29.5  1.41  0.15  1.56  108.0  106.7   1.3   

     TS%      W/L% Champion won_last won_last_3  
0  0.557  0.634146        N        N          N  
1  0.559  0.670732        N        N          N  
2  0.529  0.512195        N        N          N  
3  0.536  0.573171        N        N          N  
4  0.523  0.524390        N        N          N  


# Championship info

In [ ]:
wins = pd.read_csv("historic_dataset.csv")
wins_labels = [1 if i == "Y" else 0 for i in wins["Champion"]]


In [ ]:
from sklearn.model_selection import train_test_split


train_data, test_data, training_labels, testing_labels = train_test_split(wins, wins_labels, test_size=0.98, random_state=42)

train_data.index = range(len(train_data))
#train_labels.index = range(len(train_labels))
test_data.index = range(len(test_data))
#test_labels.index = range(len(test_labels))


training_inputs = []
for index, row in train_data.iterrows():
  won = "won" if row['won_last'] == "Y" else "didn't win"
  # Will include more in future
  train_input = f"{row['Team']} in {int(row['Year'])} has average Age: {row['Age']}, MOV: {row['MOV']}, NRtg: {row['NRtg']}, TS: {row['TS%']}, Win Percentage: {round(row['W/L%'], 3)}, and {won} last year's championship."
  training_inputs.append(train_input)


testing_inputs = []
# Loop through test_data for inputs
for index, row in test_data.iterrows():
    # Will include more in the future
    test_input = f"{row['Team']} in {int(row['Year'])} has average Age: {row['Age']}, MOV: {row['MOV']}, NRtg: {row['NRtg']}, TS: {row['TS%']}, Win Percentage: {round(row['W/L%'], 3)}, and {won} last year's championship."
    testing_inputs.append(test_input)


In [ ]:
prompt_inputs[0]

"\nDid this team positive or negative value for championship won this year? Here are three examples:\nMiami Heat in 2013 has average Age: 30.3, MOV: 7.87, NRtg: 8.6, TS: 0.588, Win Percentage: 0.805, and won last year's championship. Positive.\nLos Angeles Lakers in 2004 has average Age: 29.8, MOV: 3.9, NRtg: 4.2, TS: 0.522, Win Percentage: 0.683, and didn't win last year's championship. Negative.\nCleveland Cavaliers in 2016 has average Age: 28.1, MOV: 6.0, NRtg: 6.4, TS: 0.558, Win Percentage: 0.695, and didn't win last year's championship. Positive.\nSan Antonio Spurs in 1999 has average Age: 30.1, MOV: 8.06, NRtg: 9.0, TS: 0.523, Win Percentage: 0.74, and didn't win last year's championship.."

In [ ]:
base_prompt = """
Did this team positive or negative value for championship won this year? Here are three examples:
Miami Heat in 2013 has average Age: 30.3, MOV: 7.87, NRtg: 8.6, TS: 0.588, Win Percentage: 0.805, and won last year's championship. Championship is Positive.
Los Angeles Lakers in 2004 has average Age: 29.8, MOV: 3.9, NRtg: 4.2, TS: 0.522, Win Percentage: 0.683, and didn't win last year's championship. Championship is Negative.
Cleveland Cavaliers in 2016 has average Age: 28.1, MOV: 6.0, NRtg: 6.4, TS: 0.558, Win Percentage: 0.695, and didn't win last year's championship. Championship is Positive.
"""

prompt_inputs = [base_prompt + i + "." for i in testing_inputs]

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


count = 0
total = 0
batch_size = 64


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.generate(**t, max_new_tokens = 10).detach().cpu()
    decoded_texts = tokenizer.batch_decode(pred, skip_special_tokens=True)
    print(decoded_texts)



    for k in range(len(decoded_texts)):


      if "positive" in decoded_texts[k].lower() :
        predictions.append(1)
      elif "negative" in decoded_texts[k].lower() or "neg" :
        predictions.append(0)

      else:
        print("ERROR")



print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

['Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Negative', 'Championship is Ne

In [ ]:
base_prompt = """
Did this team positive or negative value for championship won this year? Here are three examples:
Miami Heat in 2013 has average Age: 30.3, MOV: 7.87, NRtg: 8.6, TS: 0.588, Win Percentage: 0.805, and won last year's championship. Positive.
Los Angeles Lakers in 2004 has average Age: 29.8, MOV: 3.9, NRtg: 4.2, TS: 0.522, Win Percentage: 0.683, and didn't win last year's championship. Negative.
Cleveland Cavaliers in 2016 has average Age: 28.1, MOV: 6.0, NRtg: 6.4, TS: 0.558, Win Percentage: 0.695, and didn't win last year's championship. Positive.
"""

prompt_inputs = [base_prompt + i + "." for i in testing_inputs]

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


count = 0
total = 0
batch_size = 64


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.generate(**t, max_new_tokens = 10).detach().cpu()
    decoded_texts = tokenizer.batch_decode(pred, skip_special_tokens=True)
    print(decoded_texts)



    for k in range(len(decoded_texts)):


      if "positive" in decoded_texts[k].lower() or "pos" in decoded_texts[k].lower() :
        predictions.append(1)
      else :
        predictions.append(0)





print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

['Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative']
['Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'N

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
base_prompt = """
Did this team positive or negative value for championship won this year? Here are three examples:
Miami Heat in 2013 has average Age: 30.3, MOV: 7.87, NRtg: 8.6, TS: 0.588, Win Percentage: 0.805, and won last year's championship. Positive because this team had much higher MOV, NRtg, TS, and Win Percentage compared to average mean MOV: 3.3, mean NRtg: 3.5, mean TS: 0.54, mean Win Percentage: 0.61 and won last year's championship.
Los Angeles Lakers in 2004 has average Age: 29.8, MOV: 3.9, NRtg: 4.2, TS: 0.522, Win Percentage: 0.683, and didn't win last year's championship. Negative because this team had much lower MOV, NRtg, and TS, compared to average mean MOV: 3.3, mean NRtg: 3.5, mean TS: 0.54, mean Win Percentage: 0.61.
Cleveland Cavaliers in 2016 has average Age: 28.1, MOV: 6.0, NRtg: 6.4, TS: 0.558, Win Percentage: 0.695, and didn't win last year's championship. Positive because this team had much higher MOV, NRtg, TS, and Win Percentage compared to average mean MOV: 3.3, mean NRtg: 3.5, mean TS: 0.54, mean Win Percentage: 0.61.
"""

prompt_inputs = [base_prompt + i + "." for i in testing_inputs]

"\nDid this team win this years championship? Here are three examples:\nMiami Heat in 2013 has average Age: 30.3, MOV: 7.87, NRtg: 8.6, TS: 0.588, Win Percentage: 0.805, and won last year's championship. Yes because this team had much higher MOV, NRtg, TS, and Win Percentage compared to average mean MOV: 3.3, mean NRtg: 3.5, mean TS: 0.54, mean Win Percentage: 0.61 and won last year's championship. \nLos Angeles Lakers in 2004 has average Age: 29.8, MOV: 3.9, NRtg: 4.2, TS: 0.522, Win Percentage: 0.683, and didn't win last year's championship. No because this team had much lower MOV, NRtg, TS, and Win Percentage compared to average mean MOV: 3.3, mean NRtg: 3.5, mean TS: 0.54, mean Win Percentage: 0.61.\nCleveland Cavaliers in 2016 has average Age: 28.1, MOV: 6.0, NRtg: 6.4, TS: 0.558, Win Percentage: 0.695, and didn't win last year's championship. Yes because this team had much higher MOV, NRtg, TS, and Win Percentage compared to average mean MOV: 3.3, mean NRtg: 3.5, mean TS: 0.54, m

In [ ]:

count = 0
total = 0
batch_size = 64


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.generate(**t, max_new_tokens = 10).detach().cpu()
    decoded_texts = tokenizer.batch_decode(pred, skip_special_tokens=True)
    print(decoded_texts)



    for k in range(len(decoded_texts)):


      if "positive" in decoded_texts[k].lower() or "pos" in decoded_texts[k].lower() :
        predictions.append(1)
      else :
        predictions.append(0)




print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

OutOfMemoryError: ignored

In [ ]:
count = 0
total = 0
batch_size = 16


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred1 = model.generate(**t, max_new_tokens = 10, do_sample= True, top_k = 3).detach().cpu()
    pred2 = model.generate(**t, max_new_tokens = 10, do_sample= True, top_k = 3).detach().cpu()
    pred3 = model.generate(**t, max_new_tokens = 10, do_sample= True, top_k = 3).detach().cpu()




    decoded_texts1 = tokenizer.batch_decode(pred1, skip_special_tokens=True)
    decoded_texts2 = tokenizer.batch_decode(pred2, skip_special_tokens=True)
    decoded_texts3 = tokenizer.batch_decode(pred3, skip_special_tokens=True)




    for k in range(len(decoded_texts1)):
      pos = 0


      if "yes" in decoded_texts1[k].lower():
        pos += 1

      if "yes" in decoded_texts2[k].lower():
        pos += 1

      if "yes" in decoded_texts3[k].lower():
        pos += 1

      if pos > 1:
        predictions.append(1)
      else:
        predictions.append(0)





print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

0.9324324324324325
0.030303030303030304
0.25
0.054054054054054064


# PLAYOFF PREDICTION

In [ ]:
wins = pd.read_csv("https://raw.githubusercontent.com/milesfking/NBA-Champion-Model/main/data/nba_team_advanced_data.csv")
wins = wins.drop(columns=['Losing_season', 'Arena', 'L', 'W', 'PW', 'PL', 'Attend.', 'OeFG%', 'OTOV%', 'ORB%', 'OFT/FGA',
       'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'Pace', 'FTr', '3PAr',])
wins_labels = [1 if i == "Y" else 0 for i in wins["Playof"]]




In [ ]:
wins = pd.read_csv("https://raw.githubusercontent.com/milesfking/NBA-Champion-Model/main/data/nba_team_advanced_data.csv")

train_data, test_data, training_labels, testing_labels = train_test_split(wins, wins_labels, test_size=0.98, random_state=42)

train_data.index = range(len(train_data))
#train_labels.index = range(len(train_labels))
test_data.index = range(len(test_data))
#test_labels.index = range(len(test_labels))


training_inputs = []
for index, row in train_data.iterrows():
  won = "won" if row['won_last'] == "Y" else "didn't win"
  # Will include more in future
  train_input = f"{row['Team']} in {int(row['Year'])} has average Age: {row['Age']}, MOV: {row['MOV']}, NRtg: {row['NRtg']}, TS: {row['TS%']}, Win Percentage: {round(row['W/L%'], 3)}, and {won} last year's championship."
  training_inputs.append(train_input)

,Year,Team,Age,MOV,SOS,SRS,ORtg,DRtg,NRtg,TS%,Playoffs,W/L%,Champion,won_last,won_last_3
0,1990.0,Atlanta Hawks,28.6,1.02,-0.39,0.64,113.0,111.9,1.1,0.547,N,0.500000,N,N,N
1,1990.0,Boston Celtics,30.2,3.99,-0.76,3.23,112.0,107.9,4.1,0.557,Y,0.634146,N,N,N
2,1990.0,Charlotte Hornets,25.6,-7.82,0.81,-7.00,102.0,109.9,-7.9,0.511,N,0.231707,N,N,N
3,1990.0,Chicago Bulls,26.1,3.26,-0.51,2.74,112.3,109.0,3.3,0.559,Y,0.670732,N,N,N
4,1990.0,Cleveland Cavaliers,26.1,-0.30,-0.31,-0.62,106.9,107.2,-0.3,0.529,Y,0.512195,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,2022.0,Sacramento Kings,25.8,-5.46,0.20,-5.26,109.9,115.3,-5.4,0.561,N,0.365854,N,N,N
959,2022.0,San Antonio Spurs,24.5,0.12,-0.10,0.02,112.4,112.3,0.1,0.556,N,0.414634,N,N,N
960,2022.0,Toronto Raptors,24.8,2.29,0.08,2.38,112.9,110.5,2.4,0.543,Y,0.585366,N,N,Y
961,2022.0,Utah Jazz,29.3,6.04,-0.37,5.67,116.7,110.5,6.2,0.589,Y,0.597561,N,N,N


In [ ]:
from sklearn.model_selection import train_test_split


train_data, test_data, training_labels, testing_labels = train_test_split(wins, wins_labels, test_size=0.98, random_state=42)

train_data.index = range(len(train_data))
#train_labels.index = range(len(train_labels))
test_data.index = range(len(test_data))
#test_labels.index = range(len(test_labels))


training_inputs = []
for index, row in train_data.iterrows():
  won = "won" if row['won_last'] == "Y" else "didn't win"
  # Will include more in future
  train_input = f"{row['Team']} in {int(row['Year'])} has average Age: {row['Age']}, MOV: {row['MOV']}, NRtg: {row['NRtg']}, TS: {row['TS%']}, Win Percentage: {round(row['W/L%'], 3)}, and {won} last year's championship."
  training_inputs.append(train_input)


testing_inputs = []
# Loop through test_data for inputs
for index, row in test_data.iterrows():
    # Will include more in the future
    test_input = f"{row['Team']} in {int(row['Year'])} has average Age: {row['Age']}, MOV: {row['MOV']}, NRtg: {row['NRtg']}, TS: {row['TS%']}, Win Percentage: {round(row['W/L%'], 3)}, and {won} last year's championship."
    testing_inputs.append(test_input)

In [ ]:
training_inputs

["Atlanta Hawks in 2013 has average Age: 27.1, MOV: 0.4, NRtg: 0.4, TS: 0.546, Win Percentage: 0.537, and didn't win last year's championship.",
 "Utah Jazz in 2019 has average Age: 27.3, MOV: 5.26, NRtg: 5.2, TS: 0.572, Win Percentage: 0.61, and didn't win last year's championship.",
 "Orlando Magic in 1993 has average Age: 25.8, MOV: 1.32, NRtg: 1.4, TS: 0.554, Win Percentage: 0.5, and didn't win last year's championship.",
 "Golden State Warriors in 2003 has average Age: 24.4, MOV: -1.13, NRtg: -1.2, TS: 0.526, Win Percentage: 0.463, and didn't win last year's championship.",
 "Denver Nuggets in 1993 has average Age: 24.8, MOV: -1.74, NRtg: -1.7, TS: 0.518, Win Percentage: 0.439, and didn't win last year's championship.",
 "Dallas Mavericks in 2006 has average Age: 27.5, MOV: 6.07, NRtg: 6.8, TS: 0.55, Win Percentage: 0.732, and didn't win last year's championship.",
 "San Antonio Spurs in 2001 has average Age: 29.5, MOV: 7.76, NRtg: 8.6, TS: 0.541, Win Percentage: 0.707, and didn't

[1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]

In [ ]:
base_prompt = """
Do these teams have a positive or negative value for playoff appearance? Here are three examples:
Oklahoma City Thunder has average Age: 23.2, MOV: 3.49, NRtg: 3.7, TS: 0.547, Win Percentage: 0.61. Positive.
Golden State Warriors has average Age: 26.5, MOV: 1.63, NRtg: 1.6, TS: 0.559, Win Percentage: 0.537. Positive.
Cleveland Cavaliers has average Age: 28.1, MOV: 6.0, NRtg: 6.4, TS: 0.558, Win Percentage: 0.695. Positive.
"""

prompt_inputs = [base_prompt + i + "." for i in testing_inputs]

In [ ]:

count = 0
total = 0
batch_size = 64


correct = 0

predictions = []

for i in range(0, len(prompt_inputs), batch_size):
    sentences = prompt_inputs[i:i+batch_size]
    labels = torch.tensor(testing_labels[i:i+batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding = True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.generate(**t, max_new_tokens = 10).detach().cpu()
    decoded_texts = tokenizer.batch_decode(pred, skip_special_tokens=True)
    print(decoded_texts)



    for k in range(len(decoded_texts)):


      if "positive" in decoded_texts[k].lower() or "pos" in decoded_texts[k].lower() :
        predictions.append(1)
      else :
        predictions.append(0)



print(accuracy_score(y_true=testing_labels, y_pred=predictions))
print(recall_score(y_true=testing_labels, y_pred=predictions))
print(precision_score(y_true=testing_labels, y_pred=predictions))
print(f1_score(y_true=testing_labels, y_pred=predictions))

['Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive.', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive.', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive.', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative.', 'Positive.', 'Negative', 'Positive.', 'Negative', 'Negative', 'Negative', 'Positive.', 'Negative', 'Positive.', 'Negative', 'Negative', 'Positive.', 'Positive.', 'Positive.', 'Negative', 'Negative.', 'Positive.', 'Positive.', 'Positive.', 'Negative', 'Positive.', 'Positive.', 'Negative', 'Negative', 'Negative', 'Positive.', 'Negative', 'Positive.', 'Negative', 'Negative', 'Positive.']
['Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive.', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Nega

In [ ]:
predictions

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
